# Genetic Test Expression from OMOP to IDEA

In [ ]:
import mysql.connector
import pandas as pd
import psycopg2
import random
import numpy as np
from datetime import datetime, timedelta

In [ ]:
conn = psycopg2.connect(
    dbname="omopdb",
    user="postgres",
    password="mysecretpassword",
    host="localhost",
    port="5432"
)

curOMOP = conn.cursor()
config = {
    'user': 'user', 
    'password': 'password',
    'host': '127.0.0.1',
    'database': 'idea4rc_dm',
    'raise_on_warnings': True
}

conn2 = mysql.connector.connect(**config)
curIDEA = conn2.cursor()

In [ ]:
vocabs = {
    "gene_expression_analysis_performed": 44808037,
    "gene_mutation_analysis_performed": 4038339,
    "tests_for_chromosome_translocations_performed": 4039364,
    "ngs_performed": 40485067,
    "pcr_test_performed": 40482792,
    "immunohistochemistry_performed": 4035726,
    "ct_dna_performed": 19387013,
}

columns = {
    "gene_expression_analysis_performed": "date_of_gene_expression",
    "gene_mutation_analysis_performed": "date_of_gene_mutation",
    "tests_for_chromosome_translocations_performed": "date_of_translocation",
    "ngs_performed": "date_of_ngs",
    "pcr_test_performed": "date_of_pcr",
    "immunohistochemistry_performed": "date_of_immunohistochemistry",
    "ct_dna_performed": "date_of_ct_dna"
}

selectProcOcurr = """SELECT person_id, procedure_date,  FROM procedure_occurrence 
    INNER JOIN visit_occurrence ON procedure_occurrence.visit_occurrence_id = visit_occurrence.visit_occurrence_id
    WHERE procedure_concept_id=%s"""

def insert(colName,value,date,dateCol,idLine):
    updt="UPDATE genetic_test_expression SET {colName} = %s AND {dateCol} = %s WHERE id=%s"
    curIDEA.execute(updt,(value,date,idLine))
    conn2.commit()

for column in vocabs.keys():
    curIDEA.execute(selectProcOcurr,(vocabs[column]))
    for row in curIDEA.fetchall():
        idLine = row[0]
        date = row[1]
        value = random.choice([0,1])
        insert(column,value,date,columns[column],idLine)